### Install and import Libraries

In [ ]:
! pip install arxiv python-dotenv typing openai mcp -q

In [ ]:
import arxiv
import json
import os
import dotenv
from typing import List
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()



### Tool Functions

In [ ]:
PAPRER_DIR = "papers"

In [ ]:
def search_papers(topic: str, max_results: int = 5) -> List[dict]:
    """
    Search for papers on arxiv based on topic and number of results and store the results.

    Args:
        topic: str : The topic to search for
        max_results: int : Maximum number of results to retrieve (default: 5)
    Returns:
        List of paper IDs found in the search
    """
    # Use arxiv to search for papers
    client = arxiv.Client()

    # Search for papers
    search = arxiv.Search(
        query=topic,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )

    papers = client.results(search)

    # Make directory for this topic
    path = os.path.join(PAPRER_DIR, topic.lower().replace(' ', '_'))
    os.makedirs(path, exist_ok=True)

    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # Save papers info
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)

    print(f"Results are saved in: {file_path}")
    
    return paper_ids

search_papers("computers")

In [ ]:
def extract_info(paper_id: str) -> str:
    """
    Extract information from paper based on paper id

    Args:
        paper_id: str : The paper id to extract information from
    Returns:
        str : The information extracted from the paper
    """
    
    for item in os.listdir(PAPRER_DIR):
        item_path = os.path.join(PAPRER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, 'papers_info.json')
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue
    return f"There's no saved information related to paper {paper_id}."

extract_info('2505.17021v1')

### Tool Schema

In [ ]:
tools = [
    {
        "name": "search_papers",
        "description": "Search for papers on arxiv based on topic and number of results",
        "input_schema": {
            "type": "object",
            "properties":{
                "topic": {
                    "type": "string",
                    "description": "The topic to search for"
                },
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of results to retrieve (default: 5)"
                }
            },
            "required":["topic"]
        }
    },
    {
        "name":"extract_info",
        "description":"Extract information from paper based on paper id",
        "input_schema":{
            "type":"object",
            "properties":{
                "paper_id":{
                    "type":"string",
                    "description":"The paper id to extract information from"
                }
            },
            "required":["paper_id"]
        }
    }
]

### Tool Mapping

In [ ]:
mapping_tool_function = {
    "search_papers":search_papers,
    "extract_info":extract_info
}

def execute_tool(tool_name: str, tool_args: dict) -> str:
    # Parse the JSON string if tool_args is a string
    if isinstance(tool_args, str):
        try:
            tool_args = json.loads(tool_args)
        except json.JSONDecodeError:
            return "Error: Invalid JSON arguments provided"
    
    try:
        result = mapping_tool_function[tool_name](**tool_args)
    except Exception as e:
        return f"Error executing tool: {str(e)}"
    
    if result is None:
        result = "The operation completed but didn't return any results."
    elif isinstance(result, list):
        result = ', '.join(result)
    elif isinstance(result, dict):
        result = json.dumps(result, indent=2)
    else:
        result = str(result)
    return result

In [ ]:
execute_tool("search_papers", {"topic": "computers"})

### Chatbot

In [ ]:
client = OpenAI(api_key=os.getenv("AVALAI_API_KEY"), base_url="https://api.avalai.ir/v1")

### Query Processing

In [ ]:
def process_query(query):
    # Convert tools to OpenAI format if needed
    openai_tools = []
    for tool in tools:
        openai_tools.append({
            "type": "function",
            "function": {
                "name": tool["name"],
                "description": tool.get("description", ""),
                "parameters": tool["input_schema"]
            }
        })
    
    messages = [{'role': 'user', 'content': query}]
    
    response = client.chat.completions.create(
        model='deepseek-chat',  # or your specific DeepSeek model
        max_tokens=2024,
        tools=openai_tools,
        messages=messages
    )
    
    process_query = True
    while process_query:
        assistant_message = response.choices[0].message
        assistant_content = assistant_message.content
        tool_calls = assistant_message.tool_calls

        if tool_calls:
            # Handle tool calls
            messages.append({
                "role": "assistant",
                "content": assistant_content,
                "tool_calls": tool_calls
            })
            
            for tool_call in tool_calls:
                tool_id = tool_call.id
                tool_name = tool_call.function.name
                tool_args = tool_call.function.arguments
                
                print(f"Calling tool {tool_name} with args {tool_args}")
                result = execute_tool(tool_name, tool_args)
                
                messages.append({
                    "role": "tool",
                    "content": str(result),
                    "tool_call_id": tool_id
                })
            
            # Get new response with tool results
            response = client.chat.completions.create(
                model='deepseek-chat',
                max_tokens=2024,
                tools=openai_tools,
                messages=messages
            )
        else:
            # Handle text response
            if assistant_content:
                print(assistant_content)
                process_query = False
            else:
                print("No content received from the model")
                process_query = False

### Chat Loop

In [ ]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == 'quit':
                break
    
            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

In [ ]:
chat_loop()